In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
from tqdm import tqdm
#scraping imports:
from imutils import paths
import argparse
import requests
import cv2
import os
import ctypes
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import os
import glob
import time
import re
import base64

In [39]:
DATA = r"./data/top_anime_list/UsableAnimeList.xlsx"
df = pd.read_excel(DATA, sheet_name="usable") #remember to remove duplicates
df = df["usable_title"]
#removing duplicates
df = df.drop_duplicates()
anime_list = df.tolist()

In [40]:
class scrapeAnimeImageURLs:
    def __init__(self, anime_list):
        self.downloads_folder = r"C:\Users\kaush\Downloads"
        self.img_urls_dest = r"./data/anime_img_urls"
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('--headless=new')
        self.driver = webdriver.Chrome(options=self.options)
        for anime in tqdm(anime_list, desc="AnimeList"):
            anime = self.get_saveable_animename(anime)
            self.img_dest_folder = "./data/anime_images/" + anime + "/"
            self.get_image_URLS(anime)
            url_filename = self.move_download_file(anime)
            #even encoding = 'latin-1' will work
            raw_image_urls= pd.read_csv(url_filename, encoding='ISO-8859-1', header=None)
            raw_image_urls.columns = ["image_tag"]
            processed_image_urls = raw_image_urls["image_tag"].apply(self.get_url_from_html)
            self.get_image_from_url(processed_image_urls, anime, self.img_dest_folder)
        self.driver.quit()
    
    def get_image_URLS(self, anime_name):
        print(f"Currently scraping image urls for: {anime_name}")
        self.driver.get("https://www.google.com/search?tbm=isch&q=" + anime_name)
        for _ in range(1,9): #be mindful and increase the scroll range on each loop iteration
            self.driver.execute_script(f"window.scrollBy(0,{_*10000})") # so its roughly 100 images per 10000 y-axis scroll
            time.sleep(0.5) # of which a little less than half turn out to be useful
        self.driver.execute_script(r"""
            function saveImgTagToCSV(jsonArray) {
            const csvData = jsonArray.map((element) => {
                let imgTag = '';

                if (element.outerHTML) {
                imgTag = `"${element.outerHTML.replace(/"/g, '""')}"`;
                }

                return imgTag;
            }).join('\n');

            return csvData;
            }
            //then run this:
            const imgs_rg = Array.from(document.querySelectorAll('.rg_i'));
            const imgs_grn = Array.from(document.querySelectorAll('.GRN7Gc'));
            const imgs_ll = Array.from(document.querySelectorAll('.ll3EIMe'));
            const imgs = imgs_rg.concat(imgs_grn, imgs_ll);
            img_data = saveImgTagToCSV(imgs);
            window.open('data:text/csv;charset=utf-8,' + escape(img_data))""")
        
    def move_download_file(self, anime_name):
        time.sleep(1)
        file_list = glob.glob(os.path.sep.join([self.downloads_folder, "/*"]))
        latest_file = max(file_list, key=lambda x: os.stat(x).st_mtime)
        self.created_filename = self.img_urls_dest+"/"+anime_name+".csv"
        os.replace(latest_file, self.created_filename)
        return self.created_filename

    def get_url_from_html(self, html_code):   
        match = re.search(r'<img.*?(?:src|data-src)="(.*?)".*?>', html_code)
        if match:
            source = match.group(1)
            return source #tested that this takes care of all cases.
    
    def get_image_from_url(self, url_sources, anime_name, dest_folder):
        print(f"Currently downloading images for: {anime_name}")
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        count = 0
        for source in url_sources:
            if source.startswith('data:image') and "base64" in source:
                match = re.search(r'src="data:image\/.*?;base64,(.*?)"', source)
                if match:
                    base64_string = match.group(1)
                    # Decode the base64 string
                    image_data = base64.b64decode(base64_string)
                    count += 1
                    image_name = dest_folder + anime_name + "_" + str(count).zfill(3) + ".jpg"
                    with open(image_name, 'wb') as f:
                        f.write(image_data)
            else:
                if "favicon" not in source:
                    response = requests.get(source)
                    # Check if the request was successful
                    if response.status_code == 200:
                        count += 1
                        image_name = dest_folder + anime_name + "_" + str(count).zfill(3) + ".jpg"
                        # Open the file in binary mode and write the image content
                        with open(image_name, 'wb') as file:
                            file.write(response.content)
    
    def get_saveable_animename(self, anime_name):
        saveable_anime_name = anime_name
        for ch in ["\\", "/", ":", "*", "?", "\"", "<", ">", "|"]:
              if ch in saveable_anime_name: #windows does not allow these characters in filenames
                saveable_anime_name = saveable_anime_name.replace(ch,"")
        return saveable_anime_name  

In [41]:
scraper = scrapeAnimeImageURLs(anime_list[0:50])

AnimeList:   0%|          | 0/50 [00:00<?, ?it/s]

Currently scraping image urls for: Fullmetal Alchemist Brotherhood
Currently downloading images for: Fullmetal Alchemist Brotherhood


AnimeList:   2%|▏         | 1/50 [00:43<35:33, 43.54s/it]

Currently scraping image urls for: Your Name
Currently downloading images for: Your Name


AnimeList:   4%|▍         | 2/50 [01:24<33:42, 42.14s/it]

...

AnimeList:  96%|█████████▌| 48/50 [45:04<02:30, 75.46s/it]

Currently scraping image urls for: Grave of the Fireflies
Currently downloading images for: Grave of the Fireflies


AnimeList:  98%|█████████▊| 49/50 [45:50<01:06, 66.78s/it]

Currently scraping image urls for: Hotarubi no Mori e
Currently downloading images for: Hotarubi no Mori e


AnimeList: 100%|██████████| 50/50 [46:49<00:00, 56.18s/it]


In [42]:
scraper = scrapeAnimeImageURLs(anime_list[50:100])

AnimeList:   0%|          | 0/50 [00:00<?, ?it/s]

Currently scraping image urls for: Slam Dunk
Currently downloading images for: Slam Dunk


AnimeList:   2%|▏         | 1/50 [01:12<58:51, 72.08s/it]

Currently scraping image urls for: Kamisama Hajimemashita
Currently downloading images for: Kamisama Hajimemashita


AnimeList:   4%|▍         | 2/50 [02:41<1:05:59, 82.50s/it]

...

AnimeList:  96%|█████████▌| 48/50 [53:30<02:28, 74.15s/it]

Currently scraping image urls for: Nanatsu no Taizai
Currently downloading images for: Nanatsu no Taizai


AnimeList:  98%|█████████▊| 49/50 [54:39<01:12, 72.70s/it]

Currently scraping image urls for: No Game, No Life
Currently downloading images for: No Game, No Life


AnimeList: 100%|██████████| 50/50 [56:06<00:00, 67.32s/it]


In [43]:
scraper = scrapeAnimeImageURLs(anime_list[100:150])

AnimeList:   0%|          | 0/50 [00:00<?, ?it/s]

Currently scraping image urls for: Nodame Cantabile
Currently downloading images for: Nodame Cantabile


AnimeList:   2%|▏         | 1/50 [01:30<1:14:00, 90.63s/it]

Currently scraping image urls for: Noragami Aragoto
Currently downloading images for: Noragami Aragoto


AnimeList:   4%|▍         | 2/50 [02:56<1:10:09, 87.70s/it]

...

AnimeList:  96%|█████████▌| 48/50 [50:04<01:38, 49.48s/it]

Currently scraping image urls for: Tomorrow's Joe
Currently downloading images for: Tomorrow's Joe


AnimeList:  98%|█████████▊| 49/50 [50:50<00:48, 48.49s/it]

Currently scraping image urls for: Durarara!!
Currently downloading images for: Durarara!!


AnimeList: 100%|██████████| 50/50 [51:39<00:00, 61.98s/it]


In [44]:
scraper = scrapeAnimeImageURLs(anime_list[150:200])

AnimeList:   0%|          | 0/50 [00:00<?, ?it/s]

Currently scraping image urls for: Sound! Euphonium
Currently downloading images for: Sound! Euphonium


AnimeList:   2%|▏         | 1/50 [01:00<49:16, 60.35s/it]

Currently scraping image urls for: Kiki's Delivery Service
Currently downloading images for: Kiki's Delivery Service


AnimeList:   4%|▍         | 2/50 [01:58<47:07, 58.91s/it]

...

AnimeList:  96%|█████████▌| 48/50 [41:37<01:43, 51.55s/it]

Currently scraping image urls for: Miss Kobayashi's Dragon Maid
Currently downloading images for: Miss Kobayashi's Dragon Maid


AnimeList:  98%|█████████▊| 49/50 [42:29<00:51, 51.62s/it]

Currently scraping image urls for: Girls' Last Tour
Currently downloading images for: Girls' Last Tour


AnimeList: 100%|██████████| 50/50 [43:21<00:00, 52.04s/it]


In [45]:
scraper = scrapeAnimeImageURLs(anime_list[200:])

AnimeList:   0%|          | 0/33 [00:00<?, ?it/s]

Currently scraping image urls for: Tsukimonogatari
Currently downloading images for: Tsukimonogatari


AnimeList:   3%|▎         | 1/33 [00:58<31:18, 58.69s/it]

Currently scraping image urls for: La Maison en Petits Cubes
Currently downloading images for: La Maison en Petits Cubes


AnimeList:   6%|▌         | 2/33 [01:38<24:43, 47.85s/it]

...

AnimeList:  94%|█████████▍| 31/33 [26:45<01:52, 56.47s/it]

Currently scraping image urls for: Sekaiichi Hatsukoi
Currently downloading images for: Sekaiichi Hatsukoi


AnimeList:  97%|█████████▋| 32/33 [27:37<00:55, 55.29s/it]

Currently scraping image urls for: ReLIFE
Currently downloading images for: ReLIFE


AnimeList: 100%|██████████| 33/33 [28:23<00:00, 51.61s/it]


    There are two kinds of image URLs in the image URLs file:
        - base64 encoded image URLs
        - normal image URLs
        

    Image classes:
        rg_i Q4LuWd
        GRN7Gc
        ll3IMe.CBvy7c
    Somehow we always have 0 LL images, but that's a worry for later....I'll keep it there just in case it works :D